In [1]:
import os
import sqlite3
import pandas as pd
from tqdm.notebook import tqdm
import xml.etree.ElementTree as ET

In [2]:
def convert_xml_to_readable(xml_string):
    root = ET.fromstring(xml_string.replace("&", "&amp;").replace("&amp;amp;", "&amp;"))

    readable_text = ""

    # Extract title
    title_elem = root.find(".//dc:title", {"dc": "http://purl.org/dc/elements/1.1/"})
    if title_elem is None:
        title_elem = root.find(".//official-title")
    if title_elem is not None and title_elem.text:
        title = title_elem.text.strip()
        readable_text += title + "\n\n"

    # Extract form data
    form = root.find("./form")
    if form is not None:
        congress_elem = form.find("congress")
        congress = congress_elem.text.strip() if congress_elem is not None and congress_elem.text else ""

        session_elem = form.find("session")
        session = session_elem.text.strip() if session_elem is not None and session_elem.text else ""

        legis_num_elem = form.find("legis-num")
        legis_num = legis_num_elem.text.strip() if legis_num_elem is not None and legis_num_elem.text else ""

        current_chamber_elem = form.find("current-chamber")
        current_chamber = current_chamber_elem.text.strip() if current_chamber_elem is not None and current_chamber_elem.text else ""

        action = form.find("action")
        if action is not None:
            action_date_elem = action.find("action-date")
            action_date = action_date_elem.get("date") if action_date_elem is not None else ""

            action_desc_elem = action.find("action-desc")
            action_desc = " ".join(action_desc_elem.itertext()).strip() if action_desc_elem is not None else ""

            readable_text += f"{congress}\n{session}\n{legis_num}\n\n{current_chamber}\n\n{action_date}\n\n{action_desc}\n\n"
        else:
            readable_text += f"{congress}\n{session}\n{legis_num}\n\n{current_chamber}\n\n"

        legis_type_elem = form.find("legis-type")
        legis_type = legis_type_elem.text.strip() if legis_type_elem is not None and legis_type_elem.text else ""

        official_title_elem = form.find("official-title")
        official_title = official_title_elem.text.strip() if official_title_elem is not None and official_title_elem.text else ""

        readable_text += f"{legis_type}\n\n{official_title}\n\n"

    # Extract preamble
    preamble = root.find("./preamble")
    if preamble is not None:
        preamble_text = " ".join(preamble.itertext()).strip()
        readable_text += "Preamble:\n" + preamble_text + "\n\n"

    # Extract resolution body
    resolution_body = root.find("./resolution-body")
    if resolution_body is not None:
        for section in resolution_body.findall("section"):
            section_text = " ".join(section.itertext()).strip()
            readable_text += section_text + "\n\n"
    else:
        # Extract legis-body
        legis_body = root.find("./legis-body")
        if legis_body is not None:
            for section in legis_body.findall("section"):
                section_text = " ".join(section.itertext()).strip()
                readable_text += section_text + "\n\n"

    return readable_text.strip()

In [17]:
def parse_bill_id(bill_id):
    """
    Parse the bill_id to extract congress, type, and number.
    
    Args:
    bill_id (str): A string representing the bill ID in the format BILLS-{congress}{type}{number}ih
    
    Returns:
    tuple: Returns a tuple containing the congress, type, and number extracted from the bill_id.
    """
    # Remove the 'BILLS-' prefix and 'ih' suffix
    bill_id_stripped = bill_id.replace('BILLS-', '').rstrip('ih')
    
    # Find where digits end to separate congress number from type and number
    digit_end = 0
    for char in bill_id_stripped:
        if char.isdigit():
            digit_end += 1
        else:
            break
    
    # Extract congress number
    congress = bill_id_stripped[:digit_end]
    
    # Remaining string contains type and number, need to separate them
    remaining = bill_id_stripped[digit_end:]
    
    # Extract number from the end of the string
    number = ''.join(filter(str.isdigit, remaining))
    type_start = remaining.find(number)
    
    # Extract bill type
    bill_type = remaining[:type_start]
    
    return congress, bill_type, number



def load_bill_text(db_path, bill_text_dir):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()


    # Drop the existing bill_text table if it exists (REMOVE AFTER TESTING)
    cursor.execute("DROP TABLE IF EXISTS bill_text")
    
    # Create the updated bill_text table if it doesn't exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS bill_text (
            bill_id TEXT PRIMARY KEY,
            bill_congress INTEGER,
            bill_type TEXT,
            bill_number TEXT,
            xml_content TEXT,
            last_edited_date TEXT,
            readable_text TEXT
        )
    """)

    for congress_dir in tqdm(os.listdir(bill_text_dir), desc="Congress"):
        congress_path = os.path.join(bill_text_dir, congress_dir)
        if not os.path.isdir(congress_path):
            continue
        
        bill_text_data = []

        for category_dir in tqdm(os.listdir(congress_path), desc="Category", leave=False):
            category_path = os.path.join(congress_path, category_dir)
            if not os.path.isdir(category_path):
                continue

            for bill_type_dir in tqdm(os.listdir(category_path), desc="Bill Type", leave=False):
                bill_type_path = os.path.join(category_path, bill_type_dir)
                if not os.path.isdir(bill_type_path):
                    continue

                for file_name in tqdm(os.listdir(bill_type_path), desc="Bills", leave=False):
                    if file_name.endswith('.xml'):
                        bill_file_path = os.path.join(bill_type_path, file_name)
                        with open(bill_file_path, 'r', encoding='utf-8') as file:
                            xml_content = file.read()

                        bill_id = file_name.replace('.xml', '')
                        bill_congress, bill_type, bill_number = parse_bill_id(bill_id)
                        last_edited_date = None

                        last_edited_file = file_name.replace('.xml', '-lastmod.txt')
                        last_edited_path = os.path.join(bill_type_path, last_edited_file)
                        if os.path.exists(last_edited_path):
                            with open(last_edited_path, 'r') as file:
                                last_edited_date = file.read().strip()

                        readable_text = convert_xml_to_readable(xml_content)

                        bill_text_data.append({
                            'bill_id': bill_id,
                            'bill_congress': bill_congress,
                            'bill_type': bill_type,
                            'bill_number': bill_number,
                            'xml_content': xml_content,
                            'last_edited_date': last_edited_date,
                            'readable_text': readable_text
                        })

        # Load data into a DataFrame and then into the SQLite database
        bill_text_df = pd.DataFrame(bill_text_data)
        bill_text_df.to_sql('bill_text', conn, if_exists='append', index=False, method='multi', chunksize=100)

    conn.close()

# Usage example
db_path = "data/bill_data.db"
bill_text_dir = "data/bill-text"
load_bill_text(db_path, bill_text_dir)


Congress:   0%|          | 0/6 [00:00<?, ?it/s]

Category:   0%|          | 0/2 [00:00<?, ?it/s]

Bill Type:   0%|          | 0/8 [00:00<?, ?it/s]

Bills:   0%|          | 0/230 [00:00<?, ?it/s]

Bills:   0%|          | 0/278 [00:00<?, ?it/s]

...

Bills:   0%|          | 0/268 [00:00<?, ?it/s]

Bills:   0%|          | 0/700 [00:00<?, ?it/s]

Bills:   0%|          | 0/20 [00:00<?, ?it/s]

Bills:   0%|          | 0/22 [00:00<?, ?it/s]

Bills:   0%|          | 0/138 [00:00<?, ?it/s]

In [18]:
def preview_bill_text_table(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    print("Bill Text Table Preview:")
    cursor.execute("""
        SELECT bill_id, bill_congress, bill_number, bill_type, xml_content, last_edited_date, readable_text
        FROM bill_text
        LIMIT 5
    """)
    bill_text_data = cursor.fetchall()
    for row in bill_text_data:
        print(f"Bill ID: {row[0]}")
        print(f"Bill Congress: {row[1]}")
        print(f"Bill Type: {row[3]}")
        print(f"Bill Number: {row[2]}")
        print(f"Last Edited Date: {row[5]}")
        print(f"XML Content:\n{row[4]}")
        print(f"Readable Text: {row[6]}")
        print("------------------------")

    print("\nTotal Bill Text Count:")
    cursor.execute("SELECT COUNT(*) FROM bill_text")
    total_bill_text = cursor.fetchone()[0]
    print(f"Total Bill Text Records: {total_bill_text}")

    print("\nBill Text Count by Congress:")
    cursor.execute("""
        SELECT substr(bill_id, 1, 3) as congress, COUNT(*) as count
        FROM bill_text
        GROUP BY congress
    """)
    bill_text_by_congress = cursor.fetchall()
    for row in bill_text_by_congress:
        print(f"Congress {row[0]}: {row[1]} bills")

    conn.close()

# Usage
db_path = "data/bill_data.db"
preview_bill_text_table(db_path)

Bill Text Table Preview:
Bill ID: BILLS-113hconres10ih
Bill Congress: 113
Bill Type: hconres
Bill Number: 10
Last Edited Date: 2023-01-08T17:43:15.369Z
XML Content:
<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="billres.xsl"?>
<!DOCTYPE resolution PUBLIC "-//US Congress//DTDs/res.dtd//EN" "res.dtd">
<resolution dms-id="HE1B634ADB5524EE89838587297AC47D8" key="H" public-private="public" resolution-stage="Introduced-in-House" resolution-type="house-concurrent" star-print="no-star-print">
	<metadata xmlns:dc="http://purl.org/dc/elements/1.1/">
<dublinCore>
<dc:title>113 HCON 10 IH: Supporting the goals and ideals of No Name-Calling Week in bringing attention to name-calling of all kinds and providing schools with the tools and inspiration to launch an on-going dialogue about ways to eliminate name-calling and bullying in their communities.</dc:title>
<dc:publisher>U.S. House of Representatives</dc:publisher>
<dc:date>2013-01-25</dc:date>
<dc:format>text/xml</dc:format>
<dc:la